In [16]:
import sys
!{sys.executable} -m pip install tensorflow

     |████████████████████████████████| 73.6MB 1.6MB/s eta 0:00:01    |▍                               | 860kB 1.8MB/s eta 0:00:40     |▉                               | 1.9MB 1.8MB/s eta 0:00:40     |████▋                           | 10.5MB 4.2MB/s eta 0:00:16     |███████████████████████████▎    | 62.8MB 299kB/s eta 0:00:36
     |████████████████████████████████| 3.2MB 14.1MB/s eta 0:00:01
     |████████████████████████████████| 1.4MB 14.5MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/bb/10/44230dd6bf3563b8f227dbf344c908d412ad2ff48066476672f3a72e174e/wheel-0.33.4-py2.py3-none-any.whl
     |████████████████████████████████| 2.0MB 16.7MB/s eta 0:00:01
     |████████████████████████████████| 368kB 18.3MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/4e/35/11749bf99b2d4e3cceb4d55ca22590b0d7c2c62b9de38ac4a4a7f4687421/gast-0.2.2.tar.gz
     |████████████████████████████████| 51kB 12.5MB/s eta 0:00:01
     |████████████████████████████████| 51

In [5]:
import random
import json
import gym
from gym import spaces
import pandas as pd
import numpy as np

In [ ]:
'''
Each environment must implement the following gym interface

class CustomEnv(gym.Env):
    metadata = {'render.modes':['human']}
    
    def __init__(self, arg1, arg2, ...):
        super(CustomEnv, self).__init__()
        
        self.action_space = spaces.Discrete(N_DISCRETE_ACTIONS)
        
        self.observation_space = spaces.Box(low=0,high=255,shape=(HEIGHT, WIDTH, N_CHANNELS), dype=np.uint8)
        
    def step(self,action):
        pass
        
    def reset(self):
        pass
        
    def render(self,mode='human',close=False):
        pass
            
'''

In [37]:
# stock trading environemnt for OpenAI gym

MAX_ACCOUNT_BALANCE = 2147483647
MAX_NUM_SHARES = 2147483647
MAX_SHARE_PRICE = 5000
MAX_STEPS = 20000

INITIAL_ACCOUNT_BALANCE = 10000

class StockTradingEnvironment(gym.Env):
    
    metadata = {'render.modes':['human']}
    
    def __init__(self, df):
        super(StockTradingEnvironment, self).__init__()
        self.df = df
        self.reward_range = (0, MAX_ACCOUNT_BALANCE)
        # actions of the format Buy x%, Sell x%, Hold, etc.
        self.action_space = spaces.Box(low=np.array([0,0]), high=np.array([3,1]), 
                                       dtype=np.float16)
        # prices contains the OHCL values for the last 5 prices
        self.observation_space = spaces.Box(low=0,high=1,shape=(6,6),dtype=np.float16)
        
    def _next_observation(self):
        # Get the data points for the last 5 days and scale to between 0-1
        frame = np.array([
            self.df.loc[self.current_step: self.current_step + 5, 'Open'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step + 5, 'High'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step + 5, 'Low'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step + 5, 'Close'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step + 5, 'Volume'].values / MAX_NUM_SHARES,
        ])
        
        # Append additional data and scale each value to between 0-1
        obs = np.append(frame, [[
            self.balance / MAX_ACCOUNT_BALANCE,
            self.max_net_worth / MAX_ACCOUNT_BALANCE,
            self.shares_held / MAX_NUM_SHARES,
            self.cost_basis / MAX_SHARE_PRICE,
            self.total_shares_sold / MAX_NUM_SHARES,
            self.total_sales_value / (MAX_NUM_SHARES * MAX_SHARE_PRICE),
        ]], axis=0)
        
        return obs
    
    # called any time a new environment is created or to reset an existing 
    # environment's state    
    def reset(self):
        self.balance = INITIAL_ACCOUNT_BALANCE
        self.net_worth = INITIAL_ACCOUNT_BALANCE
        self.max_net_worth = INITIAL_ACCOUNT_BALANCE
        self.shares_held = 0
        self.cost_basis = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0
        
        # set to random to give our agent more unique experiences from 
        # the same data set
        self.current_step = random.randint(0,len(self.df.loc[:,'Open'].values - 6))
        
        return self._next_observation()
    
    def _take_action(self, action):
        # set the current price to a random price within the time step
        current_price = random.uniform(
            self.df.loc[self.current_step,'Open'],
            self.df.loc[self.current_step,'Close'])
        
        action_type = action[0]
        amount = action[1]
        
        if action_type < 1:
            # buy amount % of balance in shares
            total_possible = self.balance / current_price
            shares_bought = total_possible * amount
            prev_cost = self.cost_basis * self.shares_held
            additional_cost = shares_bought * current_price
            
            self.balance -= additional_cost
            self.cost_basis = (prev_cost + additional_cost) / (self.shares_held + shares_bought)
            self.shares_held += shares_bought
        elif action_type < 2:
            # sell amount % of shares held
            shares_sold = self.shares_held * amount
            self.balance += shares_sold * current_price
            self.shares_held -= shares_sold
            self.total_shares_sold += shares_sold
            self.total_sales_value += shares_sold * current_price
        
        self.net_worth = self.balance + self.shares_held * current_price
        
        if self.net_worth > self.max_net_worth:
            self.max_net_worth = self.net_worth
        
        if self.shares_held == 0:
            self.cost_basis = 0
        
    
    def step(self,action):
        # Execute one time step with in the environment
        self._take_action(action)
        
        self.current_step += 1
        
        if self.current_step > len(self.df.loc[:,'Open'].values) - 6:
            self.current_step = 0
            
        delay_modifier = (self.current_step / MAX_STEPS)
        
        reward = self.balance * delay_modifier
        done = self.net_worth <= 0
        
        obs = self._next_observation()
        
        return obs, reward, done, {}
    
    def render(self,mode='humnan',close=False):
        # render the environment to the screen
        profit = self.net_worth - INITIAL_ACCOUNT_BALANCE
        
        print('Step: {}'.format(self.current_step))
        print('Balance: {}'.format(self.balance))
        print('Shares held: {} Total sold: {}'.format(self.shares_held, self.total_shares_sold))
        print('Avg cost for held shares: {} Total sales value: {}'.format(self.cost_basis, self.total_sales_value))
        print('Net worth: {} Max net worth: {}'.format(self.net_worth, self.total_sales_value))
        print('Profit: {}'.format(profit))

In [17]:
# Example using the above environment
# Notes:
#     in order to get stable_baselines to install, i had to run:
#     brew install mpich
import gym
import json
import datetime as dt

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

import pandas as pd


In [20]:
df = pd.read_csv('./data/AAPL.csv')
df = df.sort_values('Date')

In [38]:
# the algorithms require a vectorized environment to run
env = DummyVecEnv([lambda: StockTradingEnvironment(df)])

In [39]:
model = PPO2(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=2000)

/Users/kylecleeton/Desktop/Ryan/Tutorials/CustomGymEnvironments/venv/lib/python3.7/site-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in double_scalars


--------------------------------------
| approxkl           | 1.1571705e-06 |
| clipfrac           | 0.0           |
| explained_variance | -3.58e-07     |
| fps                | 239           |
| n_updates          | 1             |
| policy_entropy     | 2.8382168     |
| policy_loss        | -0.0001689471 |
| serial_timesteps   | 128           |
| time_elapsed       | 3.1e-06       |
| total_timesteps    | 128           |
| value_loss         | 10217958.0    |
--------------------------------------
--------------------------------------
| approxkl           | 2.0069321e-05 |
| clipfrac           | 0.0           |
| explained_variance | 5.81e-05      |
| fps                | 495           |
| n_updates          | 2             |
| policy_entropy     | 2.840177      |
| policy_loss        | -0.0009930986 |
| serial_timesteps   | 256           |
| time_elapsed       | 0.536         |
| total_timesteps    | 256           |
| value_loss         | 8032391.5     |
-------------------------

In [40]:
obs = env.reset()

In [41]:
for i in range(2000):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()

Step: 4433
Balance: 10000.0
Shares held: 0.0 Total sold: 0
Avg cost for held shares: 0 Total sales value: 0
Net worth: 10000.0 Max net worth: 0
Profit: 0.0
Step: 4434
Balance: 10000.0
Shares held: 0.0 Total sold: 0
Avg cost for held shares: 0 Total sales value: 0
Net worth: 10000.0 Max net worth: 0
Profit: 0.0
Step: 4435
Balance: 7602.782994508743
Shares held: 20.58742059651535 Total sold: 0
Avg cost for held shares: 116.44086223686577 Total sales value: 0
Net worth: 10000.0 Max net worth: 0
Profit: 0.0
Step: 4436
Balance: 4161.385683949424
Shares held: 50.483608309874235 Total sold: 0
Avg cost for held shares: 115.65366485320315 Total sales value: 0
Net worth: 9972.633443385304 Max net worth: 0
Profit: -27.366556614695583
Step: 4437
Balance: 4161.385683949424
Shares held: 50.483608309874235 Total sold: 0.0
Avg cost for held shares: 115.65366485320315 Total sales value: 0.0
Net worth: 9875.446939131209 Max net worth: 0.0
Profit: -124.55306086879136
Step: 4438
Balance: 4161.385683949424

/Users/kylecleeton/Desktop/Ryan/Tutorials/CustomGymEnvironments/venv/lib/python3.7/site-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in double_scalars


Step: 4474
Balance: 721.8885818027783
Shares held: 83.06837040975213 Total sold: 309.8285862866956
Avg cost for held shares: 110.85097884148576 Total sales value: 34297.32510331567
Net worth: 9958.539291388483 Max net worth: 34297.32510331567
Profit: -41.460708611517475
Step: 4475
Balance: 721.8885818027783
Shares held: 83.06837040975213 Total sold: 309.8285862866956
Avg cost for held shares: 110.85097884148576 Total sales value: 34297.32510331567
Net worth: 9951.93754787724 Max net worth: 34297.32510331567
Profit: -48.06245212276008
Step: 4476
Balance: 357.03894848044365
Shares held: 86.35625572307217 Total sold: 309.8285862866956
Avg cost for held shares: 110.8554293357672 Total sales value: 34297.32510331567
Net worth: 9939.808793891372 Max net worth: 34297.32510331567
Profit: -60.19120610862774
Step: 4477
Balance: 357.03894848044365
Shares held: 86.35625572307217 Total sold: 309.8285862866956
Avg cost for held shares: 110.8554293357672 Total sales value: 34297.32510331567
Net worth

Step: 4529
Balance: 0.0
Shares held: 91.88171268946266 Total sold: 560.6235975298781
Avg cost for held shares: 116.7630239341903 Total sales value: 64160.13059534006
Net worth: 9794.115562168585 Max net worth: 64160.13059534006
Profit: -205.8844378314152
Step: 4530
Balance: 9612.8300763971
Shares held: 0.0 Total sold: 652.5053102193408
Avg cost for held shares: 0 Total sales value: 73772.96067173716
Net worth: 9612.8300763971 Max net worth: 73772.96067173716
Profit: -387.16992360289987
Step: 4531
Balance: 4926.7646659205675
Shares held: 44.397014971355496 Total sold: 652.5053102193408
Avg cost for held shares: 105.54910985569491 Total sales value: 73772.96067173716
Net worth: 9612.8300763971 Max net worth: 73772.96067173716
Profit: -387.16992360289987
Step: 4532
Balance: 4926.7646659205675
Shares held: 44.397014971355496 Total sold: 652.5053102193408
Avg cost for held shares: 105.54910985569491 Total sales value: 73772.96067173716
Net worth: 9396.167029286014 Max net worth: 73772.96067

Step: 4587
Balance: 153.5457088821504
Shares held: 93.90654064300082 Total sold: 845.0935357714837
Avg cost for held shares: 102.93173745035287 Total sales value: 92825.64118675665
Net worth: 10066.963035181921 Max net worth: 92825.64118675665
Profit: 66.96303518192144
Step: 4588
Balance: 75.79287257664174
Shares held: 94.62908079295492 Total sold: 845.0935357714837
Avg cost for held shares: 102.96746138706204 Total sales value: 92825.64118675665
Net worth: 10258.866337909678 Max net worth: 92825.64118675665
Profit: 258.8663379096779
Step: 4589
Balance: 765.157592317498
Shares held: 88.3327688880002 Total sold: 851.3898476764384
Avg cost for held shares: 102.96746138706204 Total sales value: 93515.0059064975
Net worth: 10436.453326649138 Max net worth: 93515.0059064975
Profit: 436.45332664913803
Step: 4590
Balance: 765.157592317498
Shares held: 88.3327688880002 Total sold: 851.3898476764384
Avg cost for held shares: 102.96746138706204 Total sales value: 93515.0059064975
Net worth: 1040

...

Step: 955
Balance: 159.18347197681584
Shares held: 43.957873506712204 Total sold: 6115.004211749683
Avg cost for held shares: 19.831424707336954 Total sales value: 566525.5303129856
Net worth: 953.3106722784557 Max net worth: 566525.5303129856
Profit: -9046.689327721544
Step: 956
Balance: 159.18347197681584
Shares held: 43.957873506712204 Total sold: 6115.004211749683
Avg cost for held shares: 19.831424707336954 Total sales value: 566525.5303129856
Net worth: 983.0328016248866 Max net worth: 566525.5303129856
Profit: -9016.967198375114
Step: 957
Balance: 159.18347197681584
Shares held: 43.957873506712204 Total sold: 6115.004211749683
Avg cost for held shares: 19.831424707336954 Total sales value: 566525.5303129856
Net worth: 998.2669649197126 Max net worth: 566525.5303129856
Profit: -9001.733035080288
Step: 958
Balance: 61.82907413666298
Shares held: 49.154901147088445 Total sold: 6115.004211749683
Avg cost for held shares: 19.715260003946927 Total sales value: 566525.5303129856
Net wo

Step: 1008
Balance: 0.0
Shares held: 54.575194705208915 Total sold: 6228.956088211843
Avg cost for held shares: 20.45202899507312 Total sales value: 568859.7023857875
Net worth: 1258.1355508446063 Max net worth: 568859.7023857875
Profit: -8741.864449155393
Step: 1009
Balance: 0.0
Shares held: 54.575194705208915 Total sold: 6228.956088211843
Avg cost for held shares: 20.45202899507312 Total sales value: 568859.7023857875
Net worth: 1239.3197303530167 Max net worth: 568859.7023857875
Profit: -8760.680269646984
Step: 1010
Balance: 0.0
Shares held: 54.575194705208915 Total sold: 6228.956088211843
Avg cost for held shares: 20.45202899507312 Total sales value: 568859.7023857875
Net worth: 1196.6958476327143 Max net worth: 568859.7023857875
Profit: -8803.304152367285
Step: 1011
Balance: 0.0
Shares held: 54.575194705208915 Total sold: 6228.956088211843
Avg cost for held shares: 20.45202899507312 Total sales value: 568859.7023857875
Net worth: 1158.4178190780706 Max net worth: 568859.7023857875

Step: 1065
Balance: 0.0
Shares held: 54.783540209451125 Total sold: 6308.166105032224
Avg cost for held shares: 24.432975697018083 Total sales value: 570804.1364196807
Net worth: 1348.188018042514 Max net worth: 570804.1364196807
Profit: -8651.811981957486
Step: 1066
Balance: 0.0
Shares held: 54.783540209451125 Total sold: 6308.166105032224
Avg cost for held shares: 24.432975697018083 Total sales value: 570804.1364196807
Net worth: 1315.1741852947537 Max net worth: 570804.1364196807
Profit: -8684.825814705246
Step: 1067
Balance: 0.0
Shares held: 54.783540209451125 Total sold: 6308.166105032224
Avg cost for held shares: 24.432975697018083 Total sales value: 570804.1364196807
Net worth: 1313.6605178312523 Max net worth: 570804.1364196807
Profit: -8686.339482168747
Step: 1068
Balance: 0.0
Shares held: 54.783540209451125 Total sold: 6308.166105032224
Avg cost for held shares: 24.432975697018083 Total sales value: 570804.1364196807
Net worth: 1312.355578002587 Max net worth: 570804.13641968

Step: 1117
Balance: 0.0
Shares held: 52.66824138484589 Total sold: 6418.814759998255
Avg cost for held shares: 25.15519881517563 Total sales value: 573534.0491646208
Net worth: 1038.5948164898996 Max net worth: 573534.0491646208
Profit: -8961.4051835101
Step: 1118
Balance: 0.0
Shares held: 52.66824138484589 Total sold: 6418.814759998255
Avg cost for held shares: 25.15519881517563 Total sales value: 573534.0491646208
Net worth: 1048.3869806794535 Max net worth: 573534.0491646208
Profit: -8951.613019320546
Step: 1119
Balance: 18.105151633585717
Shares held: 51.77975810187037 Total sold: 6419.70324328123
Avg cost for held shares: 25.15519881517563 Total sales value: 573552.1543162544
Net worth: 1073.2520406612973 Max net worth: 573552.1543162544
Profit: -8926.747959338703
Step: 1120
Balance: 18.105151633585717
Shares held: 51.77975810187037 Total sold: 6419.70324328123
Avg cost for held shares: 25.15519881517563 Total sales value: 573552.1543162544
Net worth: 1071.4164270893425 Max net wo

Step: 1164
Balance: 0.0
Shares held: 52.44981498030642 Total sold: 6449.34450208275
Avg cost for held shares: 20.669778209261306 Total sales value: 574056.3789895027
Net worth: 835.2891951473297 Max net worth: 574056.3789895027
Profit: -9164.71080485267
Step: 1165
Balance: 0.0
Shares held: 52.44981498030642 Total sold: 6449.34450208275
Avg cost for held shares: 20.669778209261306 Total sales value: 574056.3789895027
Net worth: 841.3384109511771 Max net worth: 574056.3789895027
Profit: -9158.661589048823
Step: 1166
Balance: 0.0
Shares held: 52.44981498030642 Total sold: 6449.34450208275
Avg cost for held shares: 20.669778209261306 Total sales value: 574056.3789895027
Net worth: 847.0112383909051 Max net worth: 574056.3789895027
Profit: -9152.988761609095
Step: 1167
Balance: 0.0
Shares held: 52.44981498030642 Total sold: 6449.34450208275
Avg cost for held shares: 20.669778209261306 Total sales value: 574056.3789895027
Net worth: 829.3272311704126 Max net worth: 574056.3789895027
Profit: 